# [Module 8.0] Two Models with Inferencde Pipeline

Inout Pipeline의 Input, Output 약속


Preprocessor
CSV
    

In [73]:
import sagemaker
import pandas as pd
import numpy as np
import os
import time
import json
from time import strftime, gmtime

In [74]:
%store -r

#### 아래는 약 1분 정도가 소요 됩니다. 아래 셀의 [*] 의 표시가 [숫자] (에: [3])로 바뀔 때까지 기다려 주세요

In [75]:
from sagemaker.sklearn.estimator import SKLearn
sagemaker_session = sagemaker.Session()
from sagemaker import get_execution_role

role = get_execution_role()

script_path = 'log-preprocessing.py'
# instance_type = 'ml.m4.2xlarge'
instance_type = 'local'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    train_instance_type = instance_type
)
sklearn_preprocessor.fit({'train': s3_input_train})

Creating tmpmmkz_cuo_algo-1-8rq87_1 ... 
Attaching to tmpmmkz_cuo_algo-1-8rq87_12mdone
algo-1-8rq87_1  | 2020-08-14 01:50:08,448 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-8rq87_1  | 2020-08-14 01:50:08,450 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-8rq87_1  | 2020-08-14 01:50:08,459 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-8rq87_1  | 2020-08-14 01:50:08,597 sagemaker-containers INFO     Module log-preprocessing does not provide a setup.py. 
algo-1-8rq87_1  | Generating setup.py
algo-1-8rq87_1  | 2020-08-14 01:50:08,597 sagemaker-containers INFO     Generating setup.cfg
algo-1-8rq87_1  | 2020-08-14 01:50:08,597 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-8rq87_1  | 2020-08-14 01:50:08,597 sagemaker-containers INFO     Installing module with the following command:
algo-1-8rq87_1  | /miniconda3/bin/python -m pip install . 
algo-1-8rq87_1  |

tmpmmkz_cuo_algo-1-8rq87_1 exited with code 0
Aborting on container exit...
===== Job Complete =====


## Feature Transfomer를 사용하여 전처리된 학습 및 검증 데이타 생성 

![Transformer_Train](img/Fig2.1.transformer_train.png)

### Preprocessed Training data (Feature) 만들기

#### 아래는 약 1분 정도가 소요 됩니다. 아래 셀의 [*] 의 표시가 [숫자] (에: [4])로 바뀔 때까지 기다려 주세요

In [76]:
# 아웃풋 경로 지정
transform_train_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-train-output')
instance_type = 'local'
# instance_type = 'ml.m4.2xlarge'

# scikit_learn_inferencee_model 이름으로 전처리 학습 모델 생성
# TRANSFORM_MODE의 환경 변수는 전처리 모드라는 것을 알려 줌.
    # 추론시에는 환경 변수를 TRANSFORM_MODE": "inverse-label-transform" 설정 함.
    # 위의 두개의 과정을 분리할 수 있으나, 한개의 소스를 (preprocessor.py)를 사용하기 위해서, 환경 변수를 통해서 구분함.
scikit_learn_inferencee_model = sklearn_preprocessor.create_model(
    env={'TRANSFORM_MODE': 'feature-transform'})
# scikit_learn_inferencee_model 에서 Train Transformer 생성
transformer_train = scikit_learn_inferencee_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_train_output_path,
    accept = 'text/csv')


# Preprocess training input
transformer_train.transform(s3_input_train.config['DataSource']['S3DataSource']['S3Uri'], 
                            content_type='text/csv')
print('Waiting for transform job: ' + transformer_train.latest_transform_job.job_name)
transformer_train.wait()
preprocessed_train_path = transformer_train.output_path + transformer_train.latest_transform_job.job_name
print(preprocessed_train_path)

Attaching to tmpad_s4x6w_algo-1-rvt2j_1
algo-1-rvt2j_1  | Processing /opt/ml/code
algo-1-rvt2j_1  | Building wheels for collected packages: log-preprocessing
algo-1-rvt2j_1  |   Building wheel for log-preprocessing (setup.py) ... done
algo-1-rvt2j_1  |   Created wheel for log-preprocessing: filename=log_preprocessing-1.0.0-py2.py3-none-any.whl size=10218 sha256=9b26cd88078276593bafbe0a19a3ee3e1ae8a4bddf731e050a72ffa5b2f864b3
algo-1-rvt2j_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-s7zoi4ye/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-rvt2j_1  | Successfully built log-preprocessing
algo-1-rvt2j_1  | Installing collected packages: log-preprocessing
algo-1-rvt2j_1  | Successfully installed log-preprocessing-1.0.0
algo-1-rvt2j_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternat

algo-1-rvt2j_1  | 2020-08-14 01:50:16,917 INFO - root - Input_fn, Mode: feature_transform
algo-1-rvt2j_1  | 2020-08-14 01:50:16,926 INFO - root - predict_fn, Mode: feature_transform
algo-1-rvt2j_1  | After trainsformation
algo-1-rvt2j_1  |   (0, 0)	0.11941369588439606
algo-1-rvt2j_1  |   (0, 1)	-0.5962380254245051
algo-1-rvt2j_1  |   (0, 2)	1.744368057672484
algo-1-rvt2j_1  |   (0, 3)	0.9789570533336895
algo-1-rvt2j_1  |   (0, 4)	-0.028992907038264654
algo-1-rvt2j_1  |   (0, 5)	-0.8931854019845896
algo-1-rvt2j_1  |   (0, 6)	-0.8017032037830547
algo-1-rvt2j_1  |   (0, 7)	-1.9825286353116254
algo-1-rvt2j_1  |   (0, 8)	-1.5305589315744583
algo-1-rvt2j_1  |   (0, 9)	-0.6008796483844902
algo-1-rvt2j_1  |   (0, 10)	-0.44388847702445833
algo-1-rvt2j_1  |   (0, 24)	1.0
algo-1-rvt2j_1  |   (0, 65)	1.0
algo-1-rvt2j_1  |   (0, 67)	1.0
algo-1-rvt2j_1  |   (1, 0)	-1.852651636257096
algo-1-rvt2j_1  |   (1, 1)	-0.5962380254245051
algo-1-rvt2j_1  |   (1, 2)	0.1402835015166359
algo-1-rvt2j_1  |   (1, 3

#### Training 전처리된 학습 파일 확인

In [77]:
print(preprocessed_train_path)

s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-14-01-50-2020-08-14-01-50-11-263


In [78]:
! aws s3 ls {preprocessed_train_path} --recursive

2020-08-14 01:50:18    1054526 sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-14-01-50-2020-08-14-01-50-11-263/train.csv.out


In [79]:
preprocessed_train_path_file = os.path.join (preprocessed_train_path, 'train.csv.out')
df_pre_train = pd.read_csv(preprocessed_train_path_file)
df_pre_train.head()


,0.0,0.11941369588439606,-0.5962380254245051,1.744368057672484,0.9789570533336895,-0.028992907038264654,-0.8931854019845896,-0.8017032037830547,-1.9825286353116254,-1.5305589315744583,...,0.0.48,0.0.49,0.0.50,0.0.51,0.0.52,0.0.53,1.0.1,0.0.54,1.0.2,0.0.55
0,0.0,-1.852652,-0.596238,0.140284,-0.310405,0.970689,-0.689888,0.146389,1.232901,0.124852,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,1.181295,-0.596238,1.835130,0.185503,0.030988,-0.639063,1.568529,-0.063643,-0.846802,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.776769,-0.596238,0.216227,0.334276,0.136954,1.393914,1.394712,-0.634123,0.844596,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,-0.234547,1.508734,-0.459859,0.483049,-0.230929,0.224952,1.056954,0.921730,-0.810815,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.751486,1.218393,0.231046,-0.756723,0.516833,0.275776,1.043127,-2.138114,0.232814,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


### Preprocessed Validation data (Feature) 만들기

In [80]:
# 아웃풋 경로 지정
transform_validation_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-validation-output')
# scikit_learn_inferencee_model 에서 Validation Transformer 생성
transformer_validation = scikit_learn_inferencee_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_validation_output_path,
    accept = 'text/csv')
# Preprocess validation input
transformer_validation.transform(s3_input_validation.config['DataSource']['S3DataSource']['S3Uri'], content_type='text/csv')
print('Waiting for transform job: ' + transformer_validation.latest_transform_job.job_name)
transformer_validation.wait()
preprocessed_validation_path = transformer_validation.output_path+transformer_validation.latest_transform_job.job_name
print(preprocessed_validation_path)



Attaching to tmp3_h72zp9_algo-1-0tl73_1
algo-1-0tl73_1  | Processing /opt/ml/code
algo-1-0tl73_1  | Building wheels for collected packages: log-preprocessing
algo-1-0tl73_1  |   Building wheel for log-preprocessing (setup.py) ... done
algo-1-0tl73_1  |   Created wheel for log-preprocessing: filename=log_preprocessing-1.0.0-py2.py3-none-any.whl size=10218 sha256=03e5dff206def81203f11560df33cd37d452ee42a76c8948e49b0e3ff7c45616
algo-1-0tl73_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-ablwie8x/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-0tl73_1  | Successfully built log-preprocessing
algo-1-0tl73_1  | Installing collected packages: log-preprocessing
algo-1-0tl73_1  | Successfully installed log-preprocessing-1.0.0
algo-1-0tl73_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternat

algo-1-0tl73_1  | 2020-08-14 01:50:24,257 INFO - root - Input_fn, Mode: feature_transform
algo-1-0tl73_1  | 2020-08-14 01:50:24,262 INFO - root - predict_fn, Mode: feature_transform
algo-1-0tl73_1  | After trainsformation
algo-1-0tl73_1  |   (0, 0)	-1.3722767476585274
algo-1-0tl73_1  |   (0, 1)	1.4361483643078248
algo-1-0tl73_1  |   (0, 2)	-0.7210321088903731
algo-1-0tl73_1  |   (0, 3)	-0.3104051644736047
algo-1-0tl73_1  |   (0, 4)	-0.6407980073546213
algo-1-0tl73_1  |   (0, 5)	0.4282497227369044
algo-1-0tl73_1  |   (0, 6)	-1.7359695510006319
algo-1-0tl73_1  |   (0, 7)	-0.8415698900657912
algo-1-0tl73_1  |   (0, 8)	-0.8827893216762703
algo-1-0tl73_1  |   (0, 9)	-1.0027868214581757
algo-1-0tl73_1  |   (0, 10)	-0.44388847702445833
algo-1-0tl73_1  |   (0, 12)	1.0
algo-1-0tl73_1  |   (0, 65)	1.0
algo-1-0tl73_1  |   (0, 68)	1.0
algo-1-0tl73_1  |   (1, 0)	-1.8020858585098782
algo-1-0tl73_1  |   (1, 1)	-0.5962380254245051
algo-1-0tl73_1  |   (1, 2)	1.2442493161458343
algo-1-0tl73_1  |   (1, 3

## PCA 모델 생성

In [81]:
import boto3
import sagemaker
from sagemaker import get_execution_role

ecr_namespace = 'sagemaker-training-containers/'
prefix = 'pca'

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)

057716757052
us-east-2
arn:aws:iam::057716757052:role/service-role/AmazonSageMaker-ExecutionRole-20191128T110038
sagemaker-us-east-2-057716757052


In [82]:
! cp pca_byoc_train.py docker/code/

In [83]:
%%writefile docker/Dockerfile

FROM 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
    
# install python package
RUN pip install joblib


ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

ENV PATH="/opt/ml/code:${PATH}"

# Copy training code
COPY code/* /opt/ml/code/
 
WORKDIR /opt/ml/code

# ENTRYPOINT ["python", "pca_train.py"]
# In order to use SageMaker Env varaibles, use the statement below
ENV SAGEMAKER_PROGRAM pca_byoc_train.py

Overwriting docker/Dockerfile


In [84]:
import os
os.environ['account_id'] = account_id
os.environ['region'] = region
os.environ['ecr_repository_name'] = ecr_repository_name

In [85]:
%%sh

ACCOUNT_ID=${account_id}
REGION=${region}
REPO_NAME=${ecr_repository_name}

echo $REGION
echo $ACCOUNT_ID
echo $REPO_NAME


# Get the login command from ECR in order to pull down the Tensorflow-gpu:1.5 image
$(aws ecr get-login --registry-ids 257758044811 --region ${region} --no-include-email)



docker build -f docker/Dockerfile -t $REPO_NAME docker

docker tag $REPO_NAME $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest

$(aws ecr get-login --no-include-email --registry-ids $ACCOUNT_ID)

aws ecr describe-repositories --repository-names $REPO_NAME || aws ecr create-repository --repository-name $REPO_NAME

docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest



us-east-2
057716757052
sagemaker-training-containers/pca
Login Succeeded
Sending build context to Docker daemon  10.75kB
Step 1/8 : FROM 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
 ---> 30adb1aa9af5
Step 2/8 : RUN pip install joblib
 ---> Using cache
 ---> 26bceed11435
Step 3/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> d2cbe65e931a
Step 4/8 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 3ed49376f0e2
Step 5/8 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 03be6787c4b2
Step 6/8 : COPY code/* /opt/ml/code/
 ---> 568de0241677
Step 7/8 : WORKDIR /opt/ml/code
 ---> Running in 34ca4686ab46
Removing intermediate container 34ca4686ab46
 ---> a2af672796d8
Step 8/8 : ENV SAGEMAKER_PROGRAM pca_byoc_train.py
 ---> Running in 44bfb54e730d
Removing intermediate container 44bfb54e730d
 ---> 65b4edd687ea
Successfully built 65b4edd687ea
Successfully tagged sagemaker-training-containers/pca:latest
Login Succeeded
{
    "repositories

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [86]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

057716757052.dkr.ecr.us-east-2.amazonaws.com/sagemaker-training-containers/pca:latest


In [87]:
preprocessed_train_path_file = '{}/train.csv.out'.format(preprocessed_train_path)
preprocessed_validation_path_file = '{}/validation.csv.out'.format(preprocessed_validation_path)
print("preprocessed_train_path_file: \n", preprocessed_train_path_file)
print("preprocessed_validation_path_file: \n", preprocessed_validation_path_file)

preprocessed_train_path_file: 
 s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-14-01-50-2020-08-14-01-50-11-263/train.csv.out
preprocessed_validation_path_file: 
 s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-validation-output/sagemaker-scikit-learn-2020-08-14-01-50-2020-08-14-01-50-18-598/validation.csv.out


## PCA 학습

In [88]:
import pandas as pd

preprocessed_train_path_file = '{}/train.csv.out'.format(preprocessed_train_path)
pre_df = pd.read_csv(preprocessed_train_path_file, header=None)
print(pre_df.shape)
num_cols = pre_df.shape[1]
print("num_cols: ", num_cols)

(2333, 70)
num_cols:  70


In [89]:
import pandas as pd
# preprocessed_train_path_file = 's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-12-07-07-2020-08-12-07-07-08-229/train.csv.out'

churn_df = pd.read_csv(preprocessed_train_path_file, header=None)
churn_df.head()
train_y = churn_df.iloc[:,0]
train_X = churn_df.iloc[:,1:]

print("Shape of train_X: ", train_X.shape)
print("Shape of train_y: ", train_y.shape)

os.makedirs('./data', exist_ok =True)
np.savetxt('./data/churn-preprocessed.csv', train_X, delimiter=',',
           fmt='%1.5f'
          )

WORK_DIRECTORY = 'data'
prefix = 'Scikit-pca-custom'
train_input = sagemaker_session.upload_data(WORK_DIRECTORY,
                                            key_prefix="{}/{}".format(prefix, WORK_DIRECTORY)
                                           )
print("train_input: ", train_input)


Shape of train_X:  (2333, 69)
Shape of train_y:  (2333,)
train_input:  s3://sagemaker-us-east-2-057716757052/Scikit-pca-custom/data


In [90]:
%%time

import sagemaker

instance_type = 'local'
# instance_type = 'ml.m4.xlarge'

pca_estimator = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type= instance_type,
                                    base_job_name=prefix)

pca_estimator.set_hyperparameters(n_components= 15)

train_config = sagemaker.session.s3_input(train_input, content_type='text/csv')

pca_estimator.fit({'train': train_config})

Creating tmp_ivom6e8_algo-1-lsow7_1 ... 
Attaching to tmp_ivom6e8_algo-1-lsow7_12mdone
algo-1-lsow7_1  | 2020-08-14 01:50:31,641 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-lsow7_1  | 2020-08-14 01:50:31,643 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-lsow7_1  | 2020-08-14 01:50:31,652 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-lsow7_1  | 2020-08-14 01:50:31,653 sagemaker-containers INFO     Module pca_byoc_train does not provide a setup.py. 
algo-1-lsow7_1  | Generating setup.py
algo-1-lsow7_1  | 2020-08-14 01:50:31,653 sagemaker-containers INFO     Generating setup.cfg
algo-1-lsow7_1  | 2020-08-14 01:50:31,653 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-lsow7_1  | 2020-08-14 01:50:31,653 sagemaker-containers INFO     Installing module with the following command:
algo-1-lsow7_1  | /miniconda3/bin/python -m pip install . 
algo-1-lsow7_1  | Pr

tmp_ivom6e8_algo-1-lsow7_1 exited with code 0
Aborting on container exit...
===== Job Complete =====
CPU times: user 179 ms, sys: 40.7 ms, total: 219 ms
Wall time: 4.78 s


# Transforming Train PCA

In [91]:
import pandas as pd

preprocessed_train_path_file = '{}/train.csv.out'.format(preprocessed_train_path)
pre_df = pd.read_csv(preprocessed_train_path_file, header=None)
print(pre_df.shape)
num_cols = pre_df.shape[1]
print("num_cols: ", num_cols)

(2333, 70)
num_cols:  70


In [92]:
instance_type = 'local'
# instance_type = 'ml.m4.2xlarge'
transform_train_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-pca-train-output')

pca_model = pca_estimator.create_model(
    env={'TRANSFORM_MODE': 'feature-transform', 'LENGTH_COLS': str(num_cols)})

# scikit_learn_inferencee_model 에서 Train Transformer 생성
transformer_train = pca_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_train_output_path,
    accept = 'text/csv')


# Preprocess training input
transformer_train.transform(preprocessed_train_path_file, 
                            content_type='text/csv',                            
                           )

print('Waiting for transform job: ' + transformer_train.latest_transform_job.job_name)
transformer_train.wait()

preprocessed_pca_train_path = transformer_train.output_path + transformer_train.latest_transform_job.job_name


Attaching to tmpogi6luff_algo-1-5fdi1_1
algo-1-5fdi1_1  | Processing /opt/ml/code
algo-1-5fdi1_1  | Building wheels for collected packages: pca-byoc-train
algo-1-5fdi1_1  |   Building wheel for pca-byoc-train (setup.py) ... done
algo-1-5fdi1_1  |   Created wheel for pca-byoc-train: filename=pca_byoc_train-1.0.0-py2.py3-none-any.whl size=9371 sha256=7bd2f416a234d3668ebefb3a45844a11e634607c0573df51bc7cd9a7c766e8a5
algo-1-5fdi1_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-n9423v8i/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-5fdi1_1  | Successfully built pca-byoc-train
algo-1-5fdi1_1  | Installing collected packages: pca-byoc-train
algo-1-5fdi1_1  | Successfully installed pca-byoc-train-1.0.0
algo-1-5fdi1_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-5fdi1_

algo-1-5fdi1_1  |       0   ...   69
algo-1-5fdi1_1  | 0  0.0  ...  0.0
algo-1-5fdi1_1  | 1  0.0  ...  0.0
algo-1-5fdi1_1  | 
algo-1-5fdi1_1  | [2 rows x 70 columns]
algo-1-5fdi1_1  | os.getenv('LENGTH_COLS') :  70
algo-1-5fdi1_1  | type: os.getenv('LENGTH_COLS'):  <class 'str'>
algo-1-5fdi1_1  | type of components:  <class 'numpy.ndarray'>
algo-1-5fdi1_1  | shape of components:  (2333, 15)
algo-1-5fdi1_1  | 2020-08-14 01:50:40,065 INFO - root - predict_fn: PCA components: 
algo-1-5fdi1_1  | '[[ 0.00000000e+00 -8.23085436e-01 -1.08811131e-01 ... -6.63877079e-02
algo-1-5fdi1_1  |   -1.18330736e-02 -2.74103073e-01]
algo-1-5fdi1_1  |  [ 0.00000000e+00 -3.43474254e-01  9.14233023e-02 ... -1.93403653e-02
algo-1-5fdi1_1  |   -1.81654019e-01 -2.05599485e-03]
algo-1-5fdi1_1  |  [ 1.00000000e+00 -7.64309081e-01  1.16037618e-02 ... -2.22513681e-02
algo-1-5fdi1_1  |   -3.33721163e-04  1.38533867e-01]
algo-1-5fdi1_1  |  ...
algo-1-5fdi1_1  |  [ 0.00000000e+00 -7.35732199e-01 -1.46967771e-01 ...  5

In [93]:
print(preprocessed_pca_train_path)

s3://sagemaker-us-east-2-057716757052/Scikit-pca-custom/transformtrain-pca-train-output/pca-2020-08-14-01-50-34-180-2020-08-14-01-50-34-180


In [94]:
! aws s3 ls s3://sagemaker-us-east-2-057716757052/Scikit-pca-custom/transformtrain-pca-train-output/pca-2020-08-13-01-27-21-375-2020-08-13-01-27-21-375 --recursive

2020-08-13 01:27:28     707835 Scikit-pca-custom/transformtrain-pca-train-output/pca-2020-08-13-01-27-21-375-2020-08-13-01-27-21-375/train.csv.out.out


In [95]:
preprocessed_pca_train_path_file = '{}/train.csv.out.out'.format(preprocessed_pca_train_path)
pca_preoc_df = pd.read_csv(preprocessed_pca_train_path_file, header=None)
pca_preoc_df.head()

,0
0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 3.2 Fi..."
1,<title>500 Internal Server Error</title>
2,<h1>Internal Server Error</h1>
3,<p>The server encountered an internal error an...


## PCA Validation Transforming

In [96]:
preprocessed_validation_path

's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-validation-output/sagemaker-scikit-learn-2020-08-14-01-50-2020-08-14-01-50-18-598'

In [97]:
import pandas as pd

preprocessed_validation_path_file = '{}/validation.csv.out'.format(preprocessed_validation_path)
pre_df = pd.read_csv(preprocessed_validation_path_file, header=None)
print(pre_df.shape)
num_cols = pre_df.shape[1]
print("num_cols: ", num_cols)

(666, 70)
num_cols:  70


In [98]:


instance_type = 'local'
# instance_type = 'ml.m4.2xlarge'
transform_validation_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-pca-validation-output')

pca_model = pca_estimator.create_model(
    env={'TRANSFORM_MODE': 'feature-transform', 'LENGTH_COLS': str(num_cols)})

# scikit_learn_inferencee_model 에서 Train Transformer 생성
transformer_validation = pca_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_validation_output_path,
    accept = 'text/csv')


# Preprocess training input
transformer_validation.transform(preprocessed_validation_path_file, 
                            content_type='text/csv',                            
                           )

print('Waiting for transform job: ' + transformer_validation.latest_transform_job.job_name)
transformer_validation.wait()

preprocessed_pca_validation_path = transformer_validation.output_path + transformer_validation.latest_transform_job.job_name
print(preprocessed_pca_validation_path)

Attaching to tmp806stlsz_algo-1-o44bq_1
algo-1-o44bq_1  | Processing /opt/ml/code
algo-1-o44bq_1  | Building wheels for collected packages: pca-byoc-train
algo-1-o44bq_1  |   Building wheel for pca-byoc-train (setup.py) ... done
algo-1-o44bq_1  |   Created wheel for pca-byoc-train: filename=pca_byoc_train-1.0.0-py2.py3-none-any.whl size=9372 sha256=04c85128f58215f21130865b37f71591a6da7789016a42a12be41ba752280000
algo-1-o44bq_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-9a9o_s1o/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-o44bq_1  | Successfully built pca-byoc-train
algo-1-o44bq_1  | Installing collected packages: pca-byoc-train
algo-1-o44bq_1  | Successfully installed pca-byoc-train-1.0.0
algo-1-o44bq_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-o44bq_

algo-1-o44bq_1  | 2020-08-14 01:50:47,692 INFO - root - requested data: '      0   ...   69
algo-1-o44bq_1  | 0    0.0  ...  1.0
algo-1-o44bq_1  | 1    0.0  ...  0.0
algo-1-o44bq_1  | 2    0.0  ...  1.0
algo-1-o44bq_1  | 3    0.0  ...  0.0
algo-1-o44bq_1  | 4    0.0  ...  1.0
algo-1-o44bq_1  | ..   ...  ...  ...
algo-1-o44bq_1  | 661  0.0  ...  0.0
algo-1-o44bq_1  | 662  0.0  ...  0.0
algo-1-o44bq_1  | 663  0.0  ...  1.0
algo-1-o44bq_1  | 664  0.0  ...  0.0
algo-1-o44bq_1  | 665  0.0  ...  1.0
algo-1-o44bq_1  | 
algo-1-o44bq_1  | [666 rows x 70 columns]'
algo-1-o44bq_1  | prdict_fn: Staring
algo-1-o44bq_1  | os.getenv('TRANSFORM_MODE'):  feature-transform
algo-1-o44bq_1  | predcit_fn - TRANSFORM_MODE:  feature-transform
algo-1-o44bq_1  | 2020-08-14 01:50:47,755 INFO - root - predict_fn: input_data - '      0   ...   69
algo-1-o44bq_1  | 0    0.0  ...  1.0
algo-1-o44bq_1  | 1    0.0  ...  0.0
algo-1-o44bq_1  | 2    0.0  ...  1.0
algo-1-o44bq_1  | 3    0.0  ...  0.0
algo-1-o44bq_1  | 4  

In [99]:
preprocessed_pca_validation_path_file = '{}/validation.csv.out.out'.format(preprocessed_pca_validation_path)
pca_val_preoc_df = pd.read_csv(preprocessed_pca_validation_path_file, header=None)
pca_val_preoc_df.head()

,0
0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 3.2 Fi..."
1,<title>500 Internal Server Error</title>
2,<h1>Internal Server Error</h1>
3,<p>The server encountered an internal error an...


## Inference Pipeline <a class="anchor" id="pipeline_setup"></a>

아래 그림과 같이 위에서 생성한 전처리, 알고리즘 학습, 후처리의 세가지 모델을 가지고 1개의 단일 모델을 만들어 Inference Pipleline을 생성 합니다. <br>
**입력 데이타 가공이 없이 실제 데이타가 입력이 되면, 1개의 단일 모델을 통해서 최종적으로 예측 결과인 True, False의 결과 값이 제공 됩니다.**

![Inference-pipeline](img/Fig2.2.inference_pipeline.png)


**Machine Learning Model Pipeline (Inference Pipeline)는 create_model() 를 호출하여 만들 수 있습니다.** <br>
예를 들어 여기서는 the fitted Scikit-learn inference model, the fitted Xgboost model and the psotprocessing model 의 세가지 모델을 가지고 만듦니다.

아래는 세개 모델을 생성함. 전처리, 후처리 모델 생성시에는 환경 변수를 제공 함

In [100]:
# 전처리 모델
scikit_learn_pre_process_model = sklearn_preprocessor.create_model(
    env={'TRANSFORM_MODE': 'feature-transform'})    

# PCA 전처리 모델
pca_infer_model = pca_estimator.create_model(
    env={'TRANSFORM_MODE': 'inverse-label-transform', 'LENGTH_COLS': '69'})


In [101]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3

from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = 'churn-model-inference-pipeline-' + timestamp_prefix


sklearn_preprocessor.create_model

pipeline_model = PipelineModel(
    name = model_name,
    role = role,
    models = [
        scikit_learn_pre_process_model,
        pca_infer_model,        
    ]
)

In [102]:
%%time

instance_type='ml.t2.medium'
# instance_type='local'
endpoint_name= 'churn-model-pipeline-endpoint-' + timestamp_prefix

deployed_model = pipeline_model.deploy(
    initial_instance_count=1, 
    instance_type= instance_type, 
    endpoint_name = endpoint_name,        
    wait = True
)

-------------------!CPU times: user 311 ms, sys: 3.53 ms, total: 315 ms
Wall time: 9min 32s


In [103]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
import sagemaker
sagemaker_session = sagemaker.Session()

predictor = RealTimePredictor(
    endpoint = endpoint_name,
    sagemaker_session = sagemaker_session,
    serializer = csv_serializer,
    content_type = CONTENT_TYPE_CSV,
    accept = CONTENT_TYPE_JSON
)



In [104]:

def make_inference_format(sample):
    instance = str()
    for i, token in enumerate(sample):
        # print(token)
        if i > 0:
            instance = instance  + ',' + str(token) 
        else:
            instance = instance  +  str(token) 
    return instance


In [105]:
test_df = pd.read_csv("churn_data/batch_transform_test.csv", header=None)

for i in range(10):
    sample = test_df.iloc[i,:]
    instance = make_inference_format(sample)
    print("instance: \n", instance)

    payload = instance
    churn_result = predictor.predict(payload)
    print("Churn result?: \n", churn_result)
    print("")
    break

instance: 
 KS,186,510,400-6454,no,no,0,137.8,97,23.43,187.7,118,15.95,146.4,85,6.59,8.7,6,2.35,1


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from container-2 with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/churn-model-pipeline-endpoint-2020-08-14-01-50-48 in account 057716757052 for more information.